In [1]:
#Creating a graph to figuratively show the distributed network
class Graph:
    def __init__(self):
        self.graph = {}
        self.power_array = []
        self.thres_SINR = []
        self.awgn = []
    
    def add_connections(self, data_array):
        if data_array[0] in self.graph:
            self.graph[data_array[0]].append(data_array[1:])
        else:
            self.graph[data_array[0]] = [data_array[1:]]
    
    def print_connections(self):
        print(self.graph)

### Array Contents
* Parent Node 
* Child Node 
* Channel gain for the parent node
* Average power gain in data transmission

In [2]:
graph = Graph()
graph.power_array = [10, 20, 25, 35, 40] #the initial power ratings of the nodes in the graph
parray = graph.power_array
graph.thres_SINR = [1, 3, 4, 5, 6] #assigning threshold SINR values to each power source
graph.awgn = [3, 4, 5, 6, 7] #assigning the σ(sigma) also known as additive white gaussian noise value for each node

#giving each node of the network the other nodes to which each nodes are connected 
#along with the average internal channel gain(gii) and the external channel gain between the two nodes in communication(gij)
graph.add_connections([0, 2, 5, 6,])
graph.add_connections([0, 3, 5, 6.5])
graph.add_connections([1, 0, 6, 5.5])
graph.add_connections([2, 1, 7, 6.5])
graph.add_connections([2, 1, 7, 6.5])
graph.add_connections([2, 3, 7, 7.5])
graph.add_connections([3, 4, 8, 8.5])
graph.add_connections([4, 0, 9, 7])


In [3]:
graph.print_connections()

{0: [[2, 5, 6], [3, 5, 6.5]], 1: [[0, 6, 5.5]], 2: [[1, 7, 6.5], [1, 7, 6.5], [3, 7, 7.5]], 3: [[4, 8, 8.5]], 4: [[0, 9, 7]]}


In [4]:
graph.graph

{0: [[2, 5, 6], [3, 5, 6.5]],
 1: [[0, 6, 5.5]],
 2: [[1, 7, 6.5], [1, 7, 6.5], [3, 7, 7.5]],
 3: [[4, 8, 8.5]],
 4: [[0, 9, 7]]}

In [5]:
#attempt for refactoring the code
def summation_Gij(node):
    global parray
    sums = 0
    for i in range(len(graph.graph[node])):
            sums += ((graph.graph[node][i][2]/graph.graph[node][i][1])*parray[graph.graph[node][i][0]])
    return sums

In [6]:
#Step 2 of the algorithm
#calculating the omega value
omega =[]
def omega_function():
    global parray
    for node in graph.graph:    
        sums = 0
        #finding the value of summation of Gij
        for i in range(len(graph.graph[node])):
            sums += ((graph.graph[node][i][2]/graph.graph[node][i][1])*parray[graph.graph[node][i][0]])
        #import pdb; pdb.set_trace()
        deno = (graph.thres_SINR[node]*(sums + graph.awgn[node]/graph.graph[node][i][1]))
        value = (1+(parray[node])/deno)
        omega.append(value)
                

In [7]:
#Step 3 of the algorithm
#calulating the value for the variable that helps in providing the parametric form of the optimization problem (u)
u_array = []
def u_function():
    global parray
    for node in graph.graph:
        sums = 0
        #finding the value of summation of Gij
        for i in range(len(graph.graph[node])):
            sums += ((graph.graph[node][i][2]/graph.graph[node][i][1])*parray[graph.graph[node][i][0]])
        deno = (graph.thres_SINR[node]*(sums + graph.awgn[node]/graph.graph[node][i][1]))
        u_demo = (parray[node] + deno)**4
        u_value = (parray[node]*omega[node])**2/u_demo
        u_array.append(u_value)

In [8]:
#Step 4 of the algorithm
#finding the optimal power allocation in a distributed system for max throughput
import math
def new_power_function():
    global parray
    for node in graph.graph:
        sums = 0
        #finding the value of summation of Gij
        for i in range(len(graph.graph[node])):
            sums += ((graph.graph[node][i][2]/graph.graph[node][i][1])*parray[graph.graph[node][i][0]])
        deno = u_array[node]*(1+(graph.thres_SINR[node]*sums))**2
        new_power = omega[node]**2/deno
        #Step 5 of the algorithm
        #If the difference between the new power and the old power obtained is greated than convergence threshold
        #then the old power = new power
        #here, convergence threshold = 20
        if math.ceil(abs(new_power - parray[node]))>40:
            parray[node] = new_power
        else:
            print(f'{new_power} - {parray[node]} = {math.ceil(abs(new_power - parray[node]))}')

In [9]:
#finding the maximum power allocation for ten iterations of data communications
for i in range(10):
    omega_function()
    u_function()
    new_power_function()
print(graph.power_array)

0.2233361050799194 - 20 = 20
71.36617725354571 - 35 = 37
0.2254365919369557 - 40 = 40
41.39011447701963 - 20 = 22
202.42859430846377 - 202.42859430846377 = 0
71.36617725354571 - 35 = 37
43.503430766363365 - 40 = 4
6.561038626444226 - 6.561038626444226 = 0
41.39011447701963 - 20 = 22
202.42859430846377 - 202.42859430846377 = 0
71.36617725354571 - 35 = 37
43.503430766363365 - 40 = 4
6.561038626444226 - 6.561038626444226 = 0
41.39011447701963 - 20 = 22
202.42859430846377 - 202.42859430846377 = 0
71.36617725354571 - 35 = 37
43.503430766363365 - 40 = 4
6.561038626444226 - 6.561038626444226 = 0
41.39011447701963 - 20 = 22
202.42859430846377 - 202.42859430846377 = 0
71.36617725354571 - 35 = 37
43.503430766363365 - 40 = 4
6.561038626444226 - 6.561038626444226 = 0
41.39011447701963 - 20 = 22
202.42859430846377 - 202.42859430846377 = 0
71.36617725354571 - 35 = 37
43.503430766363365 - 40 = 4
6.561038626444226 - 6.561038626444226 = 0
41.39011447701963 - 20 = 22
202.42859430846377 - 202.42859430846

In [10]:
#Final power ratings of each of the nodes in the network 
#for providing maximum throughput rates in a distributed power allocation system
graph.power_array

[6.561038626444226, 20, 202.42859430846377, 35, 40]